## Primeiros passos

In [38]:
#Importing Packagens
import requests
import json
import pandas as pd
import sqlite3
import os
from datetime import date
import mysql.connector
import pymysql
from tqdm import tqdm
from datetime import datetime, timedelta

#========= Conexões
#Para publicar o banco de dados no looker studio, abra o ngrok e digite o comando ngrok tcp 3306

token = '?api_token=4ef0bf096ae0fdde361a40a120d18e1db1b973cc'
pipedriveURL = 'https://snovio-5a30ce.pipedrive.com/api/v1/'

def get_pipedrive(get, params=None):
    url = f"{pipedriveURL}{get}{token}"             #Building the url
    response = requests.get(url, params=params)     #using request
    return response.json()

connection = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="shalazaki",
    database="pipedrive"
)

# Criar um cursor
cursor = connection.cursor()

## 1 - Preparando Banco de Dados

In [29]:

def create_table(cursor, table_info):
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_info['name']} ({table_info['columns']})"
    cursor.execute(create_table_query)

tables_info = [
    {"name": "Deal_Stage", "columns": "id INT AUTO_INCREMENT, date_update DATETIME, deal_id INT, field_key VARCHAR(255), new_value VARCHAR(255), old_value VARCHAR(255), user_id INT, PRIMARY KEY (id, date_update, deal_id)"},
    {"name": "User", "columns": "id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), timezone_name VARCHAR(255)"},
    {"name": "Stage", "columns": "id VARCHAR(255) PRIMARY KEY, order_nr VARCHAR(255), name VARCHAR(255), pipeline_id VARCHAR(255)"},
    {"name": "Pipeline", "columns": "id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), order_nr VARCHAR(255)"},
    {"name": "Lost_reason", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Scheduled", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Lead_Score_SDR", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Lead_Score_AE", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Team", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Product", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Contact_Made", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Origin", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "Label", "columns": "id VARCHAR(255) PRIMARY KEY, label VARCHAR(255)"},
    {"name": "max_update_time ", "columns": "max_update_time  DATETIME"}
]

with connection.cursor() as cursor:
    for table_info in tables_info:
        create_table(cursor, table_info)

# Certifique-se de fazer commit para salvar as alterações
connection.commit()


# 2 - Capturando Dados

### Estrutura de Captura de Dados - Construindo tabelas Fato

In [30]:
##################################### Tabela DEAL ############################################

def create_table(cursor, table_name, columns):
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns}, page_number TEXT, PRIMARY KEY (id(255)))"
    cursor.execute(create_table_query)


def insert_data(connection, table, data_list, keys):
    with connection.cursor() as cursor:
        for data in data_list:
            # Montar a cláusula ON DUPLICATE KEY UPDATE
            update_columns = ', '.join([f"{key} = %s" for key in keys])
            on_duplicate_key_update = f" ON DUPLICATE KEY UPDATE {update_columns}"

            # Montar a query SQL completa
            sql = f"INSERT INTO {table} ({', '.join(keys)}) VALUES ({', '.join(['%s'] * len(keys))}) ON DUPLICATE KEY UPDATE {update_columns}"

            # Ajuste aqui: obter valores diretamente do dicionário, com tratamento para 'page_number'
            params = [data.get(key, '') if key != 'user_id' else data.get(key, '') for key in keys]  # Corrigido aqui

            cursor.execute(sql, params + params)  # Duplicamos os parâmetros para a parte ON DUPLICATE KEY UPDATE

    connection.commit()


deal_keys = [
    "id", "title", "owner_name", "user_id", "value", "pipeline_id", "stage_id",
    "add_time", "update_time","won_time","lost_time" ,"label", "status", "lost_reason", 
    "4399b1f494a290194a563ff8d051a2931004c32c",  # Scheduled by
    "856481b27c6718cc6e2872f07d3b7c444114182a",  # Lead Score SDR
    "8efa88d5da049834ad89cd5158238c463123d91d",  # Lead Score AE
    "53835fe5eebad73f15ab9552c0cb30d6173565ce",  # Team
    "58af6c9e65d4df2e0a1273a7bacfe8af10aac1e7",  # Company Size
    "cd45c2d856d79a012fe5f723a2669058d5d9cb4c",  # Product
    "021312270ecddc7d5167019c73b28ef262e04861",  # Linkedin Profile
    "471938728cb3d88d9ccbcf9fe43799730b7bace8",  # Country
    "39d1d775f22299ba35733d9a2f6ab73a69b7fb87",  # Job Position
    "a4d67ee86fb981de06b4f0d63be05de236747474",  # Main Goals
    "060949080c3a6c723d41c3eab8ee6389b9c6b9bf",  # Contact Made
    "5d40211a6403fecbdc3c861191f66725ac1773b8"  # Origin
]

# Criar a tabela
deal_columns = [f"{key} TEXT" for key in deal_keys]
create_table(connection.cursor(), "Deal", ', '.join(deal_columns))

#Get the last update timestamp
cursor = connection.cursor()
cursor.execute('SELECT MAX(update_time) FROM deal') #Getting the last time deal updated from SQL
timestamp_str = cursor.fetchone()[0] #The value will return a string
timestamp_pipedrive = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S') #Converting the string into date
timestamp_subtracted = timestamp_pipedrive - timedelta(hours=4) #Subtracting 4 hours to get in Brazil time zone
timestamp = timestamp_subtracted.strftime('%Y-%m-%d %H:%M:%S') #Converted to last time deal update in Brazil time zone

#timestamp = '2024-04-04 00:00:00' # USAR ESSE SOMENTE QUANDO RESETAR O BANCO DE DADOS

# Capturar dados para a tabela Deal
page_number = 0
more_pages = True

while more_pages:
    get = 'recents'
    params = {"since_timestamp":[timestamp],"limit":"500", "start":[page_number], "items":"deal"}
    deal_data = get_pipedrive(get, params)

    if deal_data['data'] is not None:
        more_pages = deal_data['additional_data']['pagination']['more_items_in_collection']
        
        with connection.cursor() as cursor:
            for deal_dictionary in deal_data['data']:
                insert_data(connection, "Deal", [deal_dictionary['data']], deal_keys) 
                cursor.execute(f'UPDATE pipedrive.Deal SET page_number = %s WHERE page_number IS NULL', (page_number,))

        
        page_number += 500
    else:
        more_pages = False



In [31]:
##################################### DEAL_STAGE ############################################

#Get the last update timestamp
cursor = connection.cursor()
cursor.execute('SELECT MAX(date_update) FROM deal_stage') #Getting the last time deal updated from SQL
timestamp_subtracted = timestamp_pipedrive - timedelta(hours=4) #Subtracting 4 hours to get in Brazil time zone
timestamp = timestamp_subtracted.strftime('%Y-%m-%d %H:%M:%S') #Converted to last time deal update in Brazil time zone

# Função para inserir registros na tabela Deal_Stage
def insert_deal_stage(params_list):
    insert_sql = """
        INSERT INTO Deal_Stage (date_update, deal_id, field_key, new_value, old_value, user_id)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        new_value = VALUES(new_value),
        old_value = VALUES(old_value),
        user_id = VALUES(user_id);
    """
    with connection.cursor() as cursor:
        for params in params_list:
            # Verificar se o registro já existe na tabela
            cursor.execute("""
                SELECT COUNT(*)
                FROM Deal_Stage
                WHERE date_update = %s
                AND deal_id = %s
                AND field_key = %s;
            """, (params[0], params[1], params[2]))
            count = cursor.fetchone()[0]

            if count == 0:
                cursor.execute(insert_sql, params)
                connection.commit()

# Capturando IDs de negócios
#retomar variavel para update_time >= {timestamp}
def get_deal_ids():
    select_deal_ids_sql = f"""
        SELECT id FROM deal
        WHERE update_time >= "{timestamp}";
    """

    with connection.cursor() as cursor:
        cursor.execute(select_deal_ids_sql)
        return [row[0] for row in cursor.fetchall()]
    
# Capturando dados de fluxo de negócios
def get_deal_flow(deal_id):
    get = f"deals/{deal_id}/flow"
    params = {"items":"dealChange","limit":"500"}
    return get_pipedrive(get,params)


def main():
    batch_size = 100  # Tamanho do lote para processamento

    deal_ids_list = get_deal_ids()
    total_deals = len(deal_ids_list)  # Obtenha o número total de deals
    batches = [deal_ids_list[i:i+batch_size] for i in range(0, len(deal_ids_list), batch_size)]

    processed_deals = 0  # Inicialize o contador de deals processados

    with tqdm(total=total_deals, desc="Processing Deals", disable=False) as pbar:
        for batch in batches:
            params_list = []

            for deal_id in batch:
                dealflow = get_deal_flow(deal_id)
                data_list = dealflow.get('data', [])
                
                if data_list is not None:
                    # Se houver dados disponíveis, prossiga com o processamento
                    filtered_results = [
                        item for item in data_list 
                        if item.get('object') == 'dealChange' and 
                        (item.get('data', {}).get('field_key') == 'stage_id' or 
                         item.get('data', {}).get('field_key') == 'status' or 
                         item.get('data', {}).get('field_key') == 'user_id' or
                         item.get('data', {}).get('field_key') == 'add_time')
                    ]
                    for result in filtered_results:
                        date_update = result['timestamp']
                        field_key = result['data']['field_key']
                        new_value = result['data']['new_value']
                        old_value = result['data']['old_value']
                        user_id = result['data']['user_id']
                        params_list.append((date_update, deal_id, field_key, new_value, old_value, user_id))
                else:
                    pass

            insert_deal_stage(params_list)
            processed_deals += len(batch)  # Atualize o contador de deals processados

            pbar.update(len(batch))  # Atualize a barra de progresso

    return "All deals processed."

if __name__ == "__main__":
    main()


Processing Deals: 100%|██████████| 3112/3112 [55:40<00:00,  1.07s/it] 


###  Estrutura de Captura de Dados - Enriquecendo tabelas Dimensão

In [39]:
# Truncate Tabelas User e Scheduled para receber novas informações
New_Values_UsersAndScheduled = [
    """
    TRUNCATE TABLE user;
    """,
    """
    TRUNCATE TABLE scheduled;
    """,
    """
    TRUNCATE TABLE max_update_time;
    """,

]

# Executar os comandos SQL
for command in New_Values_UsersAndScheduled:
    cursor.execute(command)

# Commit (confirmar) as alterações
connection.commit()


#======================================================================
#====== 1 - PARAMETROS NATIVOS:
#======================================================================  


def insert_data(connection, table, data_list, keys):
    cursor = connection.cursor()

    for data in data_list:
        columns = ', '.join(keys)
        values = ', '.join([f"'{data.get(key, '')}'" for key in keys])
        sql = f"INSERT INTO {table} ({columns}) VALUES ({values}) ON DUPLICATE KEY UPDATE id=id"
        cursor.execute(sql)

    connection.commit()

data_mappings = {
    "users": {"table": "User", "keys": ["id", "name", "timezone_name"]},
    "stages": {"table": "Stage", "keys": ["id", "order_nr", "name", "pipeline_id"]},
    "pipelines": {"table": "Pipeline", "keys": ["id", "order_nr", "name"]}
}

# Adicionando o código para outras tabelas no mesmo loop
for get, mapping in data_mappings.items():
    data_list = get_pipedrive(get)['data']
    keys = mapping["keys"]
    insert_data(connection, mapping["table"], data_list, keys)



#======================================================================
#====== 2 - PARAMETROS PERSONALIZADOS:
#======================================================================  


# Dicionários de parâmetros e tabelas
parameter_table_mapping = {
    '4399b1f494a290194a563ff8d051a2931004c32c': 'scheduled',
    '856481b27c6718cc6e2872f07d3b7c444114182a': 'lead_score_sdr',
    "8efa88d5da049834ad89cd5158238c463123d91d": 'lead_score_ae',
    '53835fe5eebad73f15ab9552c0cb30d6173565ce': 'team',
    'cd45c2d856d79a012fe5f723a2669058d5d9cb4c': 'product',
    '060949080c3a6c723d41c3eab8ee6389b9c6b9bf': 'contact_made',
    '5d40211a6403fecbdc3c861191f66725ac1773b8': 'origin',
    'label':'label',
    "lost_reason":"lost_reason"
}

table_columns_mapping = {
    'scheduled': ('id', 'label'),
    'lead_score_sdr': ('id', 'label'),
    'lead_score_ae': ('id', 'label'),
    'team': ('id', 'label'),
    'product': ('id', 'label'),
    'contact_made': ('id', 'label'),
    'origin': ('id', 'label'),
    'label': ('id','label'),
    'lost_reason': ('id','label')
}

get = 'dealFields'
deal_data = get_pipedrive(get)
cursor = connection.cursor()

for parameter, table_name in parameter_table_mapping.items():
    new_dict = None
    for field_dict in deal_data['data']:
        if 'key' in field_dict and field_dict['key'] == parameter:
            new_dict = field_dict
            break
    
    if new_dict is not None and 'options' in new_dict:
        for option in new_dict['options']:
            insert_columns = ', '.join(table_columns_mapping[table_name])
            insert_values = ', '.join([f"'{option[column]}'" for column in table_columns_mapping[table_name]])
            insert_query = f"INSERT INTO {table_name} ({insert_columns}) VALUES ({insert_values}) ON DUPLICATE KEY UPDATE id=id"
            cursor.execute(insert_query)

# Certifique-se de fazer commit para salvar as alterações
connection.commit()


# Organizando o banco de Dados

In [40]:
#=== 1 - TABELA DEAL_FILTERED ========================================

sql_commands = [
    "DROP TABLE IF EXISTS deal_filtered;",
    """
    CREATE TABLE deal_filtered AS
    SELECT * FROM deal;
    """,
    """
    ALTER TABLE deal_filtered
    CHANGE COLUMN `4399b1f494a290194a563ff8d051a2931004c32c` scheduled_by TEXT,
    CHANGE COLUMN `856481b27c6718cc6e2872f07d3b7c444114182a` lead_score_sdr TEXT,
    CHANGE COLUMN `8efa88d5da049834ad89cd5158238c463123d91d` lead_score_ae TEXT,
    CHANGE COLUMN `53835fe5eebad73f15ab9552c0cb30d6173565ce` team TEXT,
    CHANGE COLUMN `58af6c9e65d4df2e0a1273a7bacfe8af10aac1e7` company_size TEXT,
    CHANGE COLUMN `cd45c2d856d79a012fe5f723a2669058d5d9cb4c` product TEXT,
    CHANGE COLUMN `021312270ecddc7d5167019c73b28ef262e04861` linkedin_profile TEXT,
    CHANGE COLUMN `471938728cb3d88d9ccbcf9fe43799730b7bace8` country TEXT,
    CHANGE COLUMN `39d1d775f22299ba35733d9a2f6ab73a69b7fb87` job_position TEXT,
    CHANGE COLUMN `a4d67ee86fb981de06b4f0d63be05de236747474` main_goals TEXT,
    CHANGE COLUMN `060949080c3a6c723d41c3eab8ee6389b9c6b9bf` contact_made TEXT,
    CHANGE COLUMN `5d40211a6403fecbdc3c861191f66725ac1773b8` origin TEXT;
    """,
    """
    UPDATE deal_filtered
    SET 
    add_time = DATE_SUB(add_time, INTERVAL 3 HOUR),
    update_time = DATE_SUB(update_time, INTERVAL 3 HOUR),
    lost_time = DATE_SUB(lost_time, INTERVAL 3 HOUR);
    """,
    
    """
    ALTER TABLE deal_filtered
    ADD COLUMN Deal_Label VARCHAR(255);
    """
]


# Executar os comandos SQL
for command in sql_commands:
    cursor.execute(command)

# Commit (confirmar) as alterações
connection.commit()

In [34]:
    #Inserir posteriormente
    #"ALTER TABLE user ADD COLUMN SDR_ID INT;",
    #"UPDATE user u JOIN scheduled s ON u.name LIKE CONCAT('%', s.label, '%') SET u.SDR_ID = s.id;",
    #"UPDATE user SET SDR_ID = 275 WHERE name = 'Volodymyr';"

In [35]:
#=== 2 - Tabela New_deal_Stage

# Comandos SQL
sql_commands = [
    "DROP TABLE IF EXISTS new_deal_stage;",
    "CREATE TABLE new_deal_stage (date_updated DATETIME, deal_id INT, new_value INT, old_value INT);",
    "INSERT INTO new_deal_stage SELECT date_update, deal_id, new_value, old_value FROM deal_stage WHERE field_key = 'stage_id';",
    "INSERT INTO new_deal_stage (date_updated, deal_id, new_value, old_value) SELECT NULL, deal_id, old_value AS new_value, old_value FROM (SELECT deal_id, old_value FROM new_deal_stage WHERE (deal_id, date_updated) IN (SELECT deal_id, MIN(date_updated) FROM new_deal_stage GROUP BY deal_id)) AS latest_deals;",
    "ALTER TABLE new_deal_stage DROP COLUMN old_value;",
    "UPDATE new_deal_stage SET date_updated = DATE_SUB(date_updated, INTERVAL 3 HOUR);",
    "UPDATE new_deal_stage h1 JOIN (SELECT id, add_time FROM deal GROUP BY id) h2 ON h1.deal_id = h2.id SET h1.date_updated = h2.add_time WHERE h1.date_updated IS NULL;",
    "INSERT INTO new_deal_stage (deal_id, date_updated, new_value) SELECT d.id, d.add_time, d.stage_id FROM deal d LEFT JOIN new_deal_stage nds ON d.id = nds.deal_id WHERE nds.deal_id IS NULL;",

]

# Executar os comandos SQL
for command in sql_commands:
    cursor.execute(command)

# Commit (confirmar) as alterações
connection.commit()

In [41]:
#=== 3 - adaptando user e deal_filtered =============

# Comandos SQL
sql_commands = [
    """UPDATE user u
    JOIN scheduled s ON u.name LIKE CONCAT('%', s.label, '%')
    SET u.SDR_ID = s.id;""",
    
    """UPDATE user
    SET SDR_ID = 275
    WHERE name = 'Volodymyr';""",

    """
    update user
    set SDR_ID = 329
    where name = 'Nathalia Leichsnering Oliveira';
    """,
    """
    update user
    set SDR_ID = 294
    where name = 'Sofiia';
    """,
    
    """UPDATE user u
    SET u.SDR_ID = u.id
    WHERE u.SDR_ID IS NULL;""",
    
    """DELETE FROM user
    WHERE id = '15505148';""",

    """ 
    INSERT INTO max_update_time (max_update_time)
    SELECT MAX(update_time) AS max_update_time
    FROM deal_filtered;

    """,

    """
    UPDATE deal_filtered
    SET Deal_Label = (
        CASE
            WHEN label = '65' THEN 'upsell'
            WHEN label = '286' THEN 'personal email'
            WHEN label = '287' THEN 'outbound'
            ELSE 'inbound'
        END
    );
    """
]

# Executar os comandos SQL
for command in sql_commands:
    cursor.execute(command)

# Commit (confirmar) as alterações
connection.commit()
connection.close()